# 1. Preamboli

In [ ]:
import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

from utils import load_dataset, train_and_log_model

# 2. Import e versioning del dataset
Viene importato il dataset di esempio (iris oppure breast cancer, in base al parametro settato).

Questo dataset viene salvato in locale e versionato.

In [ ]:
# TODO: Parametro da settare per scegliere il dataset
example_name = "breast_cancer" # "breast_cancer" / "iris"

- *breast cancer dataset:* problema di classificazione binaria (benigno/maligno), 30 features, 569 campioni
- *iris dataset:* problema di classificazione multi-classe (setosa/versicolor/virginica), 4 features, 150 campioni

Opzionale: si può utilizzare un dataset differente, un dataset custom o un altro pubblico tipo "wine quality", basta che sia un problema di classificazione

In [ ]:
dataset_name = f"{example_name}_dataset"

X_train, X_test, y_train, y_test, dataset_version, dataset_path, df = load_dataset(dataset_name)

print(f"Dataset salvato in: {dataset_path}")
print(f"Versione del dataset: {dataset_version}")

df.head(2)

# 3. Setup MLflow Tracking

In [ ]:
# Per impostare l'URI di tracciamento
# mlflow.set_tracking_uri("")

Lasciando il paramtro di default (in mlflow.set_tracking_uri), i dati vengono salvati in locale nella cartella "mlruns"

In [ ]:
# Impostiamo il nome dell'esperimento
mlflow.set_experiment(f"{example_name}_Classification")

L'esperimento risulta come una sorta di "cartella" sulla UI mlflow per suddividere il tracking di varie runs

## Runs

Vengono effettuate 4 runs diverse per sperimentare 4 diverse combinazioni di parametri nel training del modello.

In [ ]:
D_randomforest_1 = {
    "model": "RandomForest",
    "n_estimators": 10, 
    "max_depth": 3,
    }

D_randomforest_2= {
    "model": "RandomForest",
    "n_estimators": 20, 
    "max_depth": 5,
    }

D_logistic_1 = {
    "model": "LogisticRegression",
    "max_iter": 10
    }

D_logistic_2 = {
    "model": "LogisticRegression",
    "max_iter": 15
    }

D_list = [D_randomforest_1, D_randomforest_2, D_logistic_1, D_logistic_2]

In [ ]:
MODEL_NAME = f"{example_name}_Classifier"

In [ ]:
for D in D_list:
    # Eseguiamo alcuni esperimenti con parametri diversi
    run_id = train_and_log_model(   model_name = MODEL_NAME,
                                    model_dict = D,
                                    X_train = X_train,
                                    X_test = X_test,
                                    y_train = y_train,
                                    y_test = y_test,
                                    dataset_version = dataset_version,
                                    dataset_name = dataset_name,
                                    )

# 4. Gestione dei modelli e MLflow Model Registry

In questo esempio viene ricercato il modello migliore (con un accuracy più alta) tra tutti quelli registrati sotto "MODEL_NAME".

Successivamente, al modello migliore viene effettuato un cambio di stato, settandolo come modello "in produzione".

In [ ]:
client = MlflowClient()

# 1. Prendi tutte le versioni registrate del modello
versions = client.search_model_versions(f"name='{MODEL_NAME}'")

# 2. Trova l'accuracy migliore tra i run
best_run_id = None
best_accuracy = -1.0
best_model_version = None

for v in versions:
    run_id = v.run_id
    metrics = client.get_run(run_id).data.params
    acc = metrics.get("accuracy", None)
    if acc is not None and float(acc) > best_accuracy:
        best_accuracy = float(acc)
        best_run_id = run_id
        best_model_version = v.version

print(f"Miglior modello trovato: run_id={best_run_id}, versione={best_model_version}, accuracy={best_accuracy:.4f}")

In [ ]:
# 3. Aggiorna lo stato del modello migliore (es. in Production)
if best_model_version is not None:
    client.transition_model_version_stage(
        name=MODEL_NAME,
        version=best_model_version,
        stage="Production",   # oppure "Staging"
        archive_existing_versions=True  # sposta gli altri modelli fuori da Production
    )
    print(f"Il modello versione {best_model_version} è stato promosso a Production.")

# 5. Inference
Esempio di inferenza:

Utilizzando il modello che è stato messo in produzione, viene fatta inferenza utilizzando 3 nuovi record del dataset.

In [ ]:
# Prepara nuovi dati di input (stesso schema usato in addestramento)
if example_name == "iris":
    # Qui uso l'Iris dataset come esempio
    new_data = pd.DataFrame({
        "sepal length (cm)": [5.1, 6.2, 5.9],
        "sepal width (cm)":  [3.5, 2.9, 3.0],
        "petal length (cm)": [1.4, 4.3, 5.1],
        "petal width (cm)":  [0.2, 1.3, 1.8],
    })


elif example_name == "breast_cancer":
    new_data = pd.DataFrame({
        "mean radius": [14.0, 20.0, 13.5],
        "mean texture": [20.0, 30.0, 15.0],
        "mean perimeter": [90.0, 130.0, 85.0],
        "mean area": [600.0, 1200.0, 500.0],
        "mean smoothness": [0.1, 0.15, 0.09],
        "mean compactness": [0.1, 0.2, 0.08],
        "mean concavity": [0.05, 0.1, 0.04],
        "mean concave points": [0.02, 0.05, 0.01],
        "mean symmetry": [0.2, 0.3, 0.18],
        "mean fractal dimension": [0.06, 0.07, 0.05],
        "radius error": [0.3, 0.4, 0.2],
        "texture error": [1.5, 2.0, 1.2],
        "perimeter error": [2.5, 3.5, 2.0],
        "area error": [20.0, 30.0, 15.0],
        "smoothness error": [0.005, 0.007, 0.004],
        "compactness error": [0.02, 0.03, 0.015],
        "concavity error": [0.02, 0.025, 0.01],
        "concave points error": [0.01, 0.015, 0.008],
        "symmetry error": [0.02, 0.03, 0.015],
        "fractal dimension error": [0.003, 0.004, 0.002],
        "worst radius": [16.0, 25.0, 14.5],
        "worst texture": [25.0, 40.0, 20.0],
        "worst perimeter": [110.0, 160.0, 95.0],
        "worst area": [800.0, 1500.0, 600.0],
        "worst smoothness": [0.15, 0.2, 0.12],
        "worst compactness": [0.25, 0.3, 0.2],
        "worst concavity": [0.15, 0.2, 0.1],
        "worst concave points": [0.07, 0.1, 0.05],
        "worst symmetry": [0.3, 0.4, 0.25],
        "worst fractal dimension": [0.08, 0.1, 0.07],
    })

In [ ]:
# Recupera il modello in produzione
model_uri = f"models:/{MODEL_NAME}/Production"
model = mlflow.pyfunc.load_model(model_uri)


# Esegui inferenza
predictions = model.predict(new_data)

print("Inferenza:")
print(predictions)